In [ ]:


import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
!pwd

In [ ]:
!pip install glob

In [ ]:
import glob


test = {}
train = {}
for i in ['COVID19', 'NORMAL', 'PNEUMONIA']:
    train[i] = glob.glob("../input/chest-xray-covid19-pneumonia/Data/train/{}/*.jpg".format(i))

    test[i] = glob.glob("../input/chest-xray-covid19-pneumonia/Data/test/{}/*.jpg".format(i))
    
for i in test.keys():
    print(i)
    print(len(train[i]), len(test[i]))
    print("ratio = {}".format(len(train[i])/ len(test[i])))

In [ ]:
def img_size(traintest, scantype):
    sizes = []
    for i in traintest[scantype]:
        img = cv2.imread(i,0)
        sizes.append(np.shape(img))
    return sizes

    
    


In [ ]:
X_train = []
Y_train = []
X_test = []
Y_test = []
tag = 0
#We'll have the Y as following 0 for COVID19 1 for NORMAL and 2 for  PNEUMONIA
for folder in train.keys():
    for file in train[folder]:
        img = cv2.imread(file, 0)
        img = cv2.resize(img, (64,64))
        X_train.append(img)
        Y_train.append(tag)
        
    #Since train and test got the same folders names
    for file in test[folder]:
        img = cv2.imread(file, 0)
        img = cv2.resize(img, (64,64))
        X_test.append(img)
        Y_test.append(tag)
    
    tag += 1
    

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)
    

        

In [ ]:
x_train = X_train.reshape(X_train.shape[0],64, 64,1)
x_test = X_test.reshape(X_test.shape[0], 64, 64,1)



x_train=x_train/255
x_test=x_test/255


x_train=x_train.astype(float)
x_test=x_test.astype(float)


from keras.utils import np_utils
y_train=np_utils.to_categorical(Y_train,3)
y_test=np_utils.to_categorical(Y_test,3)





In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

# Model configuration
batch_size = 40

loss_function = sparse_categorical_crossentropy
no_classes = 3
no_epochs = 200
optimizer = Adam()
verbosity = 1
num_folds = 2


(input_train, target_train), (input_test, target_test) = (x_train, y_train), (x_test,y_test)

acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((input_train, input_test), axis=0)
targets = np.concatenate((target_train, target_test), axis=0)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)
models =[]
# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets)

  model = Sequential()

  model.add(Conv2D(256,    (3,3),    strides=(1,1),    padding='same',    activation='relu', input_shape=(64,64,1)))
  model.add(Conv2D(128, (3,3),strides=(1,1), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.5))

  model.add(Conv2D(64, (3,3),strides=(1,1), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(32, (3,3),strides=(1,1), padding='same', activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  
  
  model.add(Dense(128, activation="relu"))
  model.add(Dropout(0.25))

  model.add(Dense(3,activation="softmax"))




  # Compile the model
  from tensorflow import keras
  opt = keras.optimizers.Adam(learning_rate=0.003)
  model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])


  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs, verbose=verbosity, validation_data=(inputs[test], targets[test]))

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])

  # Increase fold number
  fold_no = fold_no + 1

  models.append(model)

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print('Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print('Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

# Model configuration
batch_size = 40
#img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 3
no_epochs = 200
optimizer = Adam()
verbosity = 1
num_folds = 2

model = Sequential()

model.add(Conv2D(256,    (3,3),    strides=(1,1),    padding='same',    activation='relu', input_shape=(64,64,1)))
model.add(Conv2D(128, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

  
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))

model.add(Dense(3,activation="softmax"))




# Compile the model
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.003)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])

history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=no_epochs, verbose=verbosity, validation_data=(x_test, y_test), shuffle=True)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import NearMiss

# Model configuration
batch_size = 40
#img_width, img_height, img_num_channels = 32, 32, 3
loss_function = sparse_categorical_crossentropy
no_classes = 3
no_epochs = 200
optimizer = Adam()
verbosity = 1

training_generator = BalancedBatchGenerator(x_tain, y_train, sampler=NearMiss(), batch_size=10, random_state=42)


model = Sequential()

model.add(Conv2D(256,    (3,3),    strides=(1,1),    padding='same',    activation='relu', input_shape=(64,64,1)))
model.add(Conv2D(128, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

  
  
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.25))

model.add(Dense(3,activation="softmax"))




# Compile the model
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.003)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])




# Fit data to model
history = model.fit_generator(generator=generator1,
              epochs=no_epochs, verbose=verbosity, validation_data=(x_test, y_test))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np
from imblearn.tensorflow import balanced_batch_generator
from imblearn.over_sampling import RandomOverSampler

# Model configuration
batch_size = 40
loss_function = sparse_categorical_crossentropy
no_classes = 3
no_epochs = 200
optimizer = Adam()
verbosity = 1

oversample = RandomOverSampler(sampling_strategy='minority')

training_generator, steps = balanced_batch_generator(x_train, y_train, sampler=RandomOverSampler(), batch_size=60, random_state=42)

generator1 = ((np.reshape(x_train, (-1, 64, 64,1)), y_train) for (x_train,y_train) in training_generator)




model = Sequential()

model.add(Conv2D(256,    (3,3),    strides=(1,1),    padding='same',    activation='relu', input_shape=(64,64,1)))
model.add(Conv2D(128, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3,3),strides=(1,1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

  
  
model.add(Dense(128, activation="relu"))


model.add(Dense(3,activation="softmax"))




# Compile the model
from tensorflow import keras
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt,loss="categorical_crossentropy",metrics=["accuracy"])




# Fit data to model
history = model.fit_generator(generator=generator1, steps_per_epoch = steps,
              epochs=no_epochs, verbose=verbosity, validation_data=(x_test, y_test), shuffle=True)


In [ ]:
generator1 = ((np.reshape(x_train, (-1, 64, 64,1)), y_train) for (x_train,y_train) in training_generator)